In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from collections import Counter
import re

PATH = "Project-III/data/sherlock-holm.es_stories_plain-text_advs.txt"

# Read the text file
with open(PATH, 'r', encoding='utf-8') as file:
    text = file.read()

In [2]:
# Tokenize the text
def tokenize(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    words = text.split()
    return words

tokens = tokenize(text)
word_counts = Counter(tokens)
vocab = sorted(word_counts, key=word_counts.get, reverse=True)
word_to_index = {word: index + 1 for index, word in enumerate(vocab)}
index_to_word = {index + 1: word for index, word in enumerate(vocab)}
total_words = len(word_to_index) + 1

In [3]:
# Create input-output pairs
input_sequences = []
for line in text.split('\n'):
    token_list = [word_to_index[word] for word in tokenize(line) if word in word_to_index]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [4]:
# Pad the sequences
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array([np.pad(seq, (max_sequence_len - len(seq), 0), mode='constant') for seq in input_sequences])

In [5]:
# Split the sequences into input (X) and output (y)
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

# Convert output to one-hot encoded vectors
y = np.array(torch.nn.functional.one_hot(torch.tensor(y), num_classes=total_words))

In [6]:
# Create a custom Dataset class
class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.long)
        self.y = torch.tensor(y, dtype=torch.float)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

dataset = TextDataset(X, y)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Define the model
class NextWordPredictor(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
        super(NextWordPredictor, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        x = self.fc(x)
        return x

model = NextWordPredictor(total_words, 100, 150, total_words)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [7]:
from torch.optim.lr_scheduler import ReduceLROnPlateau  # Import ReduceLROnPlateau

epochs = 200
patience = 10  # Number of epochs to wait for improvement
current_patience = patience
best_loss = float('inf')  # Initialize best loss to a very high value
better_model = model
for epoch in range(epochs):
  # Training loop
  for i, (inputs, labels) in enumerate(dataloader):
    outputs = model(inputs)
    loss = criterion(outputs, labels.argmax(dim=1))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  # Early stopping
  if loss.item() < best_loss:  # Compare current training loss with best loss
    best_loss = loss.item()
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()} (Improved)')
    better_model = model
    current_patience = patience  # Restart patience
  else:
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')
    current_patience -= 1  # Decrement patience counter on no improvement

  # Stop training if patience is 0
  if current_patience == 0:
    print('Early stopping triggered!')
    break

'''
# Optional: Save the best model based on training loss
if best_loss != float('inf'):
  torch.save(model.state_dict(), 'best_model.pt')
'''

Epoch 1/200, Loss: 5.231538772583008 (Improved)
Epoch 2/200, Loss: 5.407001972198486
Epoch 3/200, Loss: 5.6162028312683105
Epoch 4/200, Loss: 4.924469947814941 (Improved)
Epoch 5/200, Loss: 3.9400370121002197 (Improved)
Epoch 6/200, Loss: 4.317702770233154
Epoch 7/200, Loss: 4.084334850311279
Epoch 8/200, Loss: 3.9123222827911377 (Improved)
Epoch 9/200, Loss: 3.8538107872009277 (Improved)
Epoch 10/200, Loss: 2.733553886413574 (Improved)
Epoch 11/200, Loss: 2.6732144355773926 (Improved)
Epoch 12/200, Loss: 2.7443723678588867
Epoch 13/200, Loss: 2.642007827758789 (Improved)
Epoch 14/200, Loss: 3.1721506118774414
Epoch 15/200, Loss: 2.2543506622314453 (Improved)
Epoch 16/200, Loss: 2.3420658111572266
Epoch 17/200, Loss: 2.1728837490081787 (Improved)
Epoch 18/200, Loss: 1.9896016120910645 (Improved)
Epoch 19/200, Loss: 1.5710467100143433 (Improved)
Epoch 20/200, Loss: 1.4134873151779175 (Improved)
Epoch 21/200, Loss: 1.320010781288147 (Improved)
Epoch 22/200, Loss: 1.1419868469238281 (Impr

"\n# Optional: Save the best model based on training loss\nif best_loss != float('inf'):\n  torch.save(model.state_dict(), 'best_model.pt')\n"

In [19]:
# Generate predictions
def predict_next_words(model, tokenizer, seed_text, next_words):
    for _ in range(next_words):
        token_list = [word_to_index[word] for word in tokenize(seed_text)]
        token_list = np.pad(token_list, (max_sequence_len - len(token_list), 0), mode='constant')
        token_list = torch.tensor(token_list[-max_sequence_len+1:], dtype=torch.long).unsqueeze(0)

        with torch.no_grad():
            predicted = model(token_list).argmax(dim=1).item()

        output_word = index_to_word[predicted]
        seed_text += " " + output_word

    return seed_text

seed_text = "as it seems"
next_words = 10
print(predict_next_words(better_model, word_to_index, seed_text, next_words))

as it seems to me to be a most of strong importance i
